Importing Libraries

In [ ]:
# downloading the necessaring libs
!pip install spacy
!python -m spacy download en_core_web_md
!pip install imbalanced-learn


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
import joblib
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import spacy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/kaggle/input/bst-300k/behaviour_simulation_train.csv
/kaggle/input/bst-300k-above100k/DataInLakhs (1).csv
/kaggle/input/similarity2/final.csv


In [4]:
def create_likes_classes(likes):
    if likes <= 100:
        return '0-100'
    elif 100 < likes <= 1000:
        return '100-1000'
    elif 1000 < likes <= 5000:
        return '1000-5000'
    elif 5000 < likes <= 10000:
        return '5000-10000'
    elif 10000 < likes <= 100000:
        return '10000-100000'
    else:
        return '100000-1000000'

In [5]:
# loading train data set file
data_path = '/kaggle/input/bst-300k/behaviour_simulation_train.csv'
df = pd.read_csv(data_path)



In [6]:
data_path = '/kaggle/input/bst-300k/behaviour_simulation_train.csv'
df2 = pd.read_csv(data_path)
df2['Likes_Class'] = df2['likes'].apply(create_likes_classes)

In [7]:
data_path = '/kaggle/input/bst-300k/behaviour_simulation_train.csv'
df = pd.read_csv(data_path)

# making a split for test-train
df=df[:290000]


# Load the spaCy model with word vectors
nlp = spacy.load("en_core_web_md")

# Create a new column 'Likes_Class' with classes based on the number of likes
df['Likes_Class'] = df['likes'].apply(create_likes_classes)

# Split the data into training and testing sets for classification
X_classification = df[['content', 'inferred company', 'date', 'username']]  # Include 'Date', 'Company', and 'Label'
y_classification = df['Likes_Class']

# Get word embeddings for the 'Content' column
X_text_embeddings = []
X_usr_embeddings = []
X_com_embeddings = []
X_date_embeddings = []
print(X_classification)

for content in X_classification['content']:
    doc = nlp(content)
    # Average word vectors to get a single vector for the entire text
    content_embedding = doc.vector
    X_text_embeddings.append(content_embedding)
for usr in X_classification['username']:
    doc = nlp(usr)
    # Average word vectors to get a single vector for the entire text
    content_embedding = doc.vector
    X_usr_embeddings.append(content_embedding)    
for c in X_classification['inferred company']:
    doc = nlp(c)
    # Average word vectors to get a single vector for the entire text
    content_embedding = doc.vector
    X_com_embeddings.append(content_embedding)     
    
for c in X_classification['date']:
    doc = nlp(c)
    # Average word vectors to get a single vector for the entire text
    content_embedding = doc.vector
    X_date_embeddings.append(content_embedding)     

X_text_embeddings_df = pd.DataFrame(X_text_embeddings)
X_usr_embeddings_df = pd.DataFrame(X_usr_embeddings)
X_com_embeddings_df = pd.DataFrame(X_com_embeddings)
X_date_embeddings_df = pd.DataFrame(X_date_embeddings)

# Combine embeddings with other features

# X_combined = pd.concat([X_classification.drop('content', axis=1), X_text_embeddings_df], axis=1)
# X_combined2 = pd.concat([X_combined.drop('date', axis=1), X_date_embeddings_df], axis=1)
X_combined3 = pd.concat([ X_text_embeddings_df, X_usr_embeddings_df, X_com_embeddings_df, X_date_embeddings_df], axis=1)
# X_combined = pd.concat([X_combined.drop('inferred company', axis=1), X_com_embeddings_df], axis=1)
# X_combined = pd.concat([X_combined.drop('username', axis=1), X_usr_embeddings_df], axis=1)

# oversampling the data
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_combined3, y_classification)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the classifier
classifier = RandomForestClassifier(n_estimators=200,max_depth=50,class_weight='balanced', random_state=46)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)
joblib.dump(classifier, 'random_forest_classifier_bst-300k.joblib')
# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Classification Accuracy: {accuracy * 100:.2f}%')



                                                  content  inferred company  \
0       Spend your weekend morning with a Ham, Egg, an...       tim hortons   
1       Watch rapper <mention> freestyle for over an H...       independent   
2       Canadian Armenian community demands ban on mil...               cbc   
3       1st in Europe to be devastated by COVID-19, It...          williams   
4       Congratulations to Pauletha Butts of <mention>...       independent   
...                                                   ...               ...   
289995  Cowboys fans, respond with a picture of how yo...           cameron   
289996  The Czech Jet TAKES OFF!\n\nCongratulations <m...  turkish airlines   
289997  Taylor Swift's second surprise album of 2020, ...               cnn   
289998  [TOMORROW] Mo G &amp; <mention> are celebratin...              sabc   
289999  ✈💪\nJuntos Somos Más Grandes!\n\nWe're proud t...  turkish airlines   

                       date         username  
0   

Code to predict on the test split we did before

In [8]:
def predict_likes_class(index, dataframe, text_model, usr_model, com_model, date_model, classifier):
    # Extract the relevant information for the given index
    content = dataframe.loc[index, 'content']
    username = dataframe.loc[index, 'username']
    company = dataframe.loc[index, 'inferred company']
    date = dataframe.loc[index, 'date']

    # Get word embeddings for the input features
    content_embedding = text_model(content).vector
    usr_embedding = usr_model(username).vector
    com_embedding = com_model(company).vector
    date_embedding = date_model(date).vector

    # Combine the embeddings with other features
    input_features = np.concatenate([content_embedding, usr_embedding, com_embedding, date_embedding], axis=None)

    # Make the prediction using the trained classifier
    predicted_class = classifier.predict([input_features])[0]

    return predicted_class

# Example usage:
# Replace 'index_to_predict' with the actual index you want to predict
index_to_predict=290000
final=100
for i in range(index_to_predict,index_to_predict+final): 
    predicted_class_label = predict_likes_class(i, df2, nlp, nlp, nlp, nlp, classifier)

    print(f"Predicted Class Label for Index {index_to_predict}: {predicted_class_label}\n actual likes are {df2.iloc[i]['likes']}" )


Predicted Class Label for Index 290000: 100-1000
 actual likes are 183
Predicted Class Label for Index 290000: 0-100
 actual likes are 0
Predicted Class Label for Index 290000: 100-1000
 actual likes are 662
Predicted Class Label for Index 290000: 0-100
 actual likes are 1
Predicted Class Label for Index 290000: 100-1000
 actual likes are 1864
Predicted Class Label for Index 290000: 100-1000
 actual likes are 58
Predicted Class Label for Index 290000: 0-100
 actual likes are 7
Predicted Class Label for Index 290000: 100-1000
 actual likes are 324
Predicted Class Label for Index 290000: 100-1000
 actual likes are 954
Predicted Class Label for Index 290000: 100-1000
 actual likes are 5632
Predicted Class Label for Index 290000: 0-100
 actual likes are 55
Predicted Class Label for Index 290000: 0-100
 actual likes are 0
Predicted Class Label for Index 290000: 0-100
 actual likes are 4
Predicted Class Label for Index 290000: 100-1000
 actual likes are 100
Predicted Class Label for Index 29